<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/innovaciones/blob/main/propuesta/reduccion_dim_clasificacion_regresion/0_intro_python/dia_1/.ipynb_checkpoints/dia_1-checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/innovaciones/blob/main/propuesta/reduccion_dim_clasificacion_regresion/0_intro_python/dia_1/.ipynb_checkpoints/dia_1-checkpoint.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Día 14: XGBoost

[XGBoost](https://www.kaggle.com/alexisbcook/xgboost?utm_medium=email&utm_source=gamma&utm_campaign=thirty-days-of-ml&utm_content=day-14)

### Objetivo:

Aprender a crear y optimizar modelos con aumento de gradiente.

## [Video de apoyo]() 

Este método domina muchas competiciones de Kaggle y logra resultados de vanguardia en una variedad de conjuntos de datos.

## Introducción

Durante gran parte de este curso, ha realizado predicciones con el método de bosque aleatorio, que logra un mejor rendimiento que un solo árbol de decisión simplemente promediando las predicciones de muchos árboles de decisión.

Nos referimos al método de bosque aleatorio como un "método de conjunto".

Por definición, los métodos de conjunto combinan las predicciones de varios modelos (por ejemplo, varios árboles, en el caso de bosques aleatorios).

A continuación, aprenderemos sobre otro método de conjunto llamado aumento de gradiente.

## Gradient Boosting

## Aumento de gradiente

El aumento de gradiente es un método que pasa por ciclos para agregar modelos de forma iterativa a un conjunto.

Comienza por inicializar el conjunto con un solo modelo, cuyas predicciones pueden ser bastante ingenuas.

(Incluso si sus predicciones son tremendamente inexactas, las adiciones posteriores al conjunto abordarán esos errores).

Luego, comenzamos el ciclo:

Primero, usamos el conjunto actual para generar predicciones para cada observación en el conjunto de datos. 

Para hacer una predicción, agregamos las predicciones de todos los modelos en el conjunto.

Estas predicciones se utilizan para calcular una función de pérdida (como el error cuadrático medio, por ejemplo).

Luego, usamos la función de pérdida para ajustar un nuevo modelo que se agregará al conjunto.

Específicamente, determinamos los parámetros del modelo para que la adición de este nuevo modelo al conjunto reduzca la pérdida.

### Nota al margen:   
el "gradiente" en "aumento de gradiente" se refiere al hecho de que usaremos el descenso de gradiente en la función de pérdida para determinar los parámetros en este nuevo modelo.

Finalmente, agregamos el nuevo modelo al conjunto, y repetimos el proceso. 

<img src = 'https://i.imgur.com/MvCGENh.png'> 

## Ejemplo

Comenzamos cargando los datos de entrenamiento y validación en `X_train`, `X_valid`, `y_train,` e `y_valid`.

### Cargar la librería para procesamiento de datos 

In [2]:
import pandas as pd

### Leer los datos

In [3]:
data = pd.read_csv('melb_data.csv')

## Selecciona un subconjunto de predictores

In [4]:
cols_to_use = ['Rooms', 'Distance', 'Landsize','BuildingArea', 'YearBuilt']
X = data[cols_to_use]

## Seleccione un objetivo

In [5]:
y = data.Price

## Separar los datos en conjuntos de entrenamiento y validación

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

En este ejemplo, trabajará con la biblioteca `XGBoost`.

`XGBoost` significa aumento de gradiente extremo, que es una implementación de aumento de gradiente con varias características adicionales centradas en el rendimiento y la velocidad.

(Scikit-learn tiene otra versión de aumento de gradiente, pero XGBoost tiene algunas ventajas técnicas).

En la siguiente celda de código, importamos la API scikit-learn para XGBoost (xgboost.XGBRegressor).

Esto nos permite construir y ajustar un modelo tal como lo haríamos en scikit-learn.

Como verá en el resultado, la clase `XGBRegressor` tiene muchos parámetros ajustables, ¡los aprenderá pronto!

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

También hacemos predicciones y evaluamos el modelo.

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Error absoluto medio: " + 
      str(mean_absolute_error(predictions, y_valid)))

## Ajuste de parámetros

XGBoost tiene algunos parámetros que pueden afectar drásticamente la precisión y la velocidad de entrenamiento. 

Los primeros parámetros que debe comprender son:

`n_estimators`

`n_estimators` especifica cuántas veces pasar por el ciclo de modelado descrito anteriormente.

Es igual a la cantidad de modelos que incluimos en el conjunto.

* Un valor demasiado bajo provoca un ajuste insuficiente, lo que conduce a predicciones inexactas 
  tanto en los datos de entrenamiento como en los de prueba.  
* Un valor demasiado alto provoca un sobreajuste, lo que genera predicciones precisas en los datos   
  de entrenamiento, pero predicciones inexactas en los datos de prueba (que es lo que nos importa).

Los valores típicos oscilan entre 100 y 1000, aunque esto depende mucho del parámetro `learning_rate` que se analiza a continuación.

Aquí está el código para establecer el número de modelos en el conjunto:

In [ ]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

### early_stopping_rounds

`early_stopping_rounds` ofrece una manera de encontrar automáticamente el valor ideal para `n_estimators`.

La detención anticipada hace que el modelo deje de iterar cuando la puntuación de validación deja de mejorar, incluso si no estamos en la parada difícil para `n_estimators`.

Es inteligente establecer un valor alto para `n_estimators` y luego usar `early_stopping_rounds` para encontrar el momento óptimo para dejar de iterar.

Dado que la probabilidad aleatoria a veces causa una sola ronda en la que las puntuaciones de validación no mejoran, debe especificar un número de cuántas rondas de deterioro directo permitir antes de detenerse.

Establecer `early_stopping_rounds = 5` es una opción razonable.

En este caso, nos detenemos después de 5 rondas seguidas de puntajes de validación en deterioro.

Cuando se utiliza `early_stopping_rounds`, también debe reservar algunos datos para calcular los puntajes de validación; esto se hace configurando el parámetro `eval_set`.

Podemos modificar el ejemplo anterior para incluir una parada anticipada:

In [ ]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

Si luego desea ajustar un modelo con todos sus datos, establezca `n_estimators` en el valor que considere óptimo cuando se ejecute con parada anticipada.

### learning_rate

En lugar de obtener predicciones simplemente sumando las predicciones de cada modelo de componente, podemos multiplicar las predicciones de cada modelo por un número pequeño (conocido como tasa de aprendizaje) antes de agregarlas.

Esto significa que cada árbol que agregamos al conjunto nos ayuda menos.

Por lo tanto, podemos establecer un valor más alto para `n_estimators` sin sobreajustar.

Si utilizamos la parada anticipada, el número apropiado de árboles se determinará automáticamente.

En general, una tasa de aprendizaje pequeña y una gran cantidad de estimadores producirán modelos XGBoost más precisos, aunque también llevará más tiempo entrenar el modelo, ya que realiza más iteraciones a lo largo del ciclo. 

De forma predeterminada, XGBoost establece `learning_rate = 0.1`.

Modificar el ejemplo anterior para cambiar la tasa de aprendizaje produce el siguiente código:

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

`n_jobs`

En conjuntos de datos más grandes donde el tiempo de ejecución es una consideración, puede usar el paralelismo para construir sus modelos más rápido.

Es común establecer el parámetro `n_jobs` igual al número de núcleos en su máquina.

En conjuntos de datos más pequeños, esto no ayudará.

El modelo resultante no será mejor, por lo que la microoptimización para el tiempo de adaptación no suele ser más que una distracción.

Pero es útil en grandes conjuntos de datos en los que, de lo contrario, pasaría mucho tiempo esperando durante el comando de ajuste.

Aquí está el ejemplo modificado:

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

## Conclusion

`XGBoost` es una biblioteca de software líder para trabajar con datos tabulares estándar (el tipo de datos que almacena en Pandas `DataFrames`, a diferencia de tipos de datos más exóticos como imágenes y videos).

Con un cuidadoso ajuste de parámetros, puede entrenar modelos de alta precisión.

## Tu turno

Entrena tu propio modelo con XGBoost en el [próximo ejercicio](https://www.kaggle.com/marcocanas/exercise-xgboost/edit)

En este ejercicio, utilizará sus nuevos conocimientos para entrenar un modelo con aumento de gradiente (gradient boosting).

## Setup

Las preguntas a continuación le darán comentarios sobre su trabajo.

Ejecute la siguiente celda para configurar el sistema de retroalimentación.

## Configurar verificación de código

In [ ]:
#import os
#if not os.path.exists("../input/train.csv"):
#    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
#    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
#from learntools.core import binder
#binder.bind(globals())
#from learntools.ml_intermediate.ex6 import *
#print("Setup Complete")

Trabajará con el conjunto de datos del [Concurso de precios de vivienda para usuarios de Kaggle Learn](https://www.kaggle.com/c/home-data-for-ml-course) del ejercicio anterior.

<img src = 'https://i.imgur.com/lTJVG4e.png'>

Ejecute la siguiente celda de código sin cambios para cargar los conjuntos de entrenamiento y validación en `X_train`, `X_valid`, `y_train` e `y_valid`.

El conjunto de prueba se carga en `X_test`.

<!--
def ocultar línea de texto
-->

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

## Leer los datos

In [ ]:
X = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

## Elimina filas con un objetivo faltante, separa el objetivo de los predictores

In [ ]:
X.dropna(axis = 0, subset = ['SalePrice'], inplace = True)
y = X.SalePrice              
X.drop(['SalePrice'], axis = 1, inplace = True)

## Dividir los datos en conjunto de validación  y conjunto de entrenamiento

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, 
                                                                train_size=0.8, 
                                                                test_size=0.2,
                                                                random_state=0)

### "Cardinalidad" significa la cantidad de valores únicos en una columna

### Selecciona columnas categóricas con cardinalidad relativamente baja (conveniente pero arbitraria)

In [ ]:
low_cardinality_cols = [cname for cname in X_train_full.columns 
                        if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

## Seleccionar columnas numéricas

In [ ]:
numeric_cols = [cname for cname in X_train_full.columns 
                if X_train_full[cname].dtype in ['int64', 'float64']]

## Mantener solo las columnas seleccionadas

In [ ]:
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

## One-hot codifica los datos (para acortar el código, usamos pandas)

In [ ]:
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join = 'left', axis = 1)
# Construir la tupla (X_train, X_valid)
X_train, X_test = X_train.align(X_test, join = 'left', axis = 1)
# Construir la tupla (X_train, X_test) 

In [ ]:
import numpy as np 
import pandas as pd 

A = pd.DataFrame(np.array([[1,2],[3,4]]), columns=['A','B'])
B = pd.DataFrame(np.array([[5,6],[7,8]]), columns= ['A','B'])
A.align(B, join = 'outer', axis = 1) 

## Paso 1: modelo de construcción

### Parte A

En este paso, creará y entrenará su primer modelo con aumento de gradiente.

Comience configurando `my_modelo_1` en un modelo `XGBoost`.

Utilice la clase `XGBRegressor` y establezca la semilla aleatoria en 0 (` random_state = 0`).

Deje todos los demás parámetros como predeterminados.

Luego, ajusta el modelo a los datos de entrenamiento en `X_train` e `y_train`.

In [ ]:
from xgboost import XGBRegressor

## Definir el modelo

In [ ]:
my_model_1 = XGBRegressor(random_state = 0) # Your code here

## Encajar el modelo

In [ ]:
my_model_1.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False) # Your code here

# Check your answer
# step_1.a.check()

## Parte B

Establezca `predictions_1` en las predicciones del modelo para los datos de validación.

Recuerde que los atributos de validación se almacenan en `X_valid`.

In [ ]:
predictions_1 = my_model_1.predict(X_valid)

## Part C

Finally, use the `mean_absolute_error()` function to calculate the mean absolute error (MAE) corresponding to the predictions for the validation set. 

Recall that the labels for the validation data are stored in `y_valid`.

In [ ]:
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1,y_valid) # Your code here

# Uncomment to print MAE
# print("Mean Absolute Error:" , mae_1)

# Check your answer
# step_1.c.check()

## Paso 2: Mejora el modelo

Ahora que ha entrenado un modelo predeterminado como línea de base, es hora de modificar los parámetros para ver si puede obtener un mejor rendimiento.

Comience configurando `my_model_2` en un modelo `XGBoost`, usando la clase `XGBRegressor`.

Use what you learned in the previous tutorial to figure out how to change the default parameters (like `n_estimators` and `learning_rate`) to get better results.

Utilice lo que aprendió en el tutorial anterior para descubrir cómo cambiar los parámetros predeterminados (como `n_estimators` y `learning_rate`) para obtener mejores resultados.

Luego, ajusta el modelo a los datos de entrenamiento en `X_train` e `y_train`.

Establezca `predictions_2` en las predicciones del modelo para los datos de validación.

Recuerde que los atributos de validación se almacenan en `X_valid`.

Finalmente, use la función `mean_absolute_error()` para calcular el error absoluto medio (MAE) correspondiente a las predicciones en el conjunto de validación.

Recuerde que las etiquetas de los datos de validación se almacenan en `y_valid`.

In order for this step to be marked correct, your model in `my_model_2` must attain lower MAE than the model in `my_model_1`.

Para que este paso se marque como correcto, su modelo en `my_model_2` debe alcanzar un MAE más bajo que el modelo en `my_model_1`. 

In [ ]:
# Define the model
my_model_2 = XGBRegressor(n_estimators = 500, learning_rate = 0.05) # Your code here

# Fit the model
my_model_2.fit(X_train, y_train) # Your code here

# Get predictions
predictions_2 = my_model_2.predict(X_valid)  # Your code here

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid) # Your code here

# Uncomment to print MAE
# print("Mean Absolute Error:" , mae_2)

# Check your answer
# step_2.check()

In [ ]:
mae_1, mae_2 

### Otra forma de solución:

In [ ]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
my_model_2.fit(X_train, y_train)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)
print("Mean Absolute Error:" , mae_2)

## Paso 3: Rompe el modelo

En este paso, creará un modelo que funciona peor que el modelo original en el Paso 1.

Esto le ayudará a desarrollar su intuición sobre cómo establecer parámetros.

Incluso puede descubrir que accidentalmente obtiene un mejor rendimiento, lo que en última instancia es un buen problema y una valiosa experiencia de aprendizaje.

Comience configurando `my_model_3` en un modelo` XGBoost`, usando la clase `XGBRegressor`.

Use lo que aprendió en el tutorial anterior para descubrir cómo cambiar los parámetros predeterminados (como `n_stimators` y `learning_rate`) para diseñar un modelo para obtener un MAE alto.

Luego, ajusta el modelo a los datos de entrenamiento en `X_train` e `y_train`.

Set `predictions_3` to the model's predictions for the validation data. 

Recall that the validation features are stored in `X_valid`.

Finally, use the `mean_absolute_error()` function to calculate the mean absolute error (MAE) corresponding to the predictions on the validation set. 

Recall that the labels for the validation data are stored in `y_valid`.

In order for this step to be marked correct, your model in `my_model_3` must attain higher MAE than the model in `my_model_1`.

In [ ]:
# Define the model
my_model_3 = XGBRegressor(n_estimators = 1) 

# Fit the model
my_model_3.fit(X_train, y_train) # Your code here

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3,y_valid)

# Uncomment to print MAE
# print("Mean Absolute Error:" , mae_3)

# Check your answer
# step_3.check()

## Sigue adelante

Continúe aprendiendo sobre la [fuga de datos](https://www.kaggle.com/alexisbcook/data-leakage).

Este es un tema importante para que lo comprenda un científico de datos, ¡y tiene el potencial de arruinar sus modelos de maneras sutiles y peligrosas!

# Fuga de datos

En este tutorial, aprenderá qué es la fuga de datos y cómo prevenirla.

Si no sabe cómo prevenirlo, las fugas aparecerán con frecuencia y arruinarán sus modelos de manera sutil y peligrosa.

Entonces, este es uno de los conceptos más importantes para los científicos de datos en práctica.

In [ ]:
!jt -l 

In [ ]:
!jt -t onedork 